In [1]:
import ROOT as r
import numpy as np
from ROOT import TFile, TTree

Welcome to JupyROOT 6.22/02


In [2]:
def thetaPhi():#
    '''returns random theta and phi values'''
    theta = np.random.uniform(0,2*np.pi)
    phi = np.random.uniform(0,np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz

def vector(g, theta, phi, mag, mass):
    '''given a tlorentzvector g the cartesian components are set with the spherical components and the mass'''
    Px, Py, Pz = momenta(mag, theta, phi)
    g.SetPxPyPzE(Px, Py, Pz, mass)

In [3]:
#defining mass of particles
wMass = 80.3

eMass = 0.0005

nMass = 0

# setting up particles
w = r.TLorentzVector()
#stationary case
w.SetPxPyPzE(0,0,0, wMass)

e = r.TLorentzVector()
n = r.TLorentzVector()

In [4]:
def components(num):
    '''given a number of events wanted, returns arrays of the components of the electron and neutrinos'''
    
    #initialising arrays for components
    ePx = np.zeros(num)
    ePy = np.zeros(num)
    ePz = np.zeros(num)
    eE = np.zeros(num)

    nPx = np.zeros(num)
    nPy = np.zeros(num)
    nPz = np.zeros(num)
    nE = np.zeros(num)
    
    # looping over number of events and filling component arrays
    for i in range(num):
        #making e components
        thetae, phie = thetaPhi()
        vector(e, thetae, phie, np.random.uniform(0,80), eMass)
        
        ePx[i] = e.Px()
        ePy[i] = e.Py()
        ePz[i] = e.Pz()
        eE[i] = e.E()
               
        #making neutrino components
        n = w - e
        n.SetE(nMass)
               
        nPx[i] = n.Px()
        nPy[i] = n.Py()
        nPz[i] = n.Pz()
        nE[i] = n.E()
        
    return ePx, ePy, ePz, eE, nPx, nPy, nPz, nE

In [5]:
file = r.TFile("Tree.root", "recreate")

In [6]:
# making 10 events worth of components
ePx, ePy, ePz, eE, nPx, nPy, nPz, nE = components(10)

In [7]:
eventsArr = np.zeros(8) # [electron[4],neutrino[4]]

In [8]:
t = r.TTree("t", "fourVectorTree")
t.Branch("events",eventsArr,"eventsArray[8]/D")

In [9]:
for i in range(len(ePx)):
    eventsArr[0] = ePx[i]
    eventsArr[1] = ePy[i]
    eventsArr[2] = ePz[i]
    eventsArr[3] = eE[i]
    
    eventsArr[4] = nPx[i]
    eventsArr[5] = nPy[i]
    eventsArr[6] = nPz[i]
    eventsArr[7] = nE[i]
    t.Fill()

In [10]:
t.Scan("events")

80

***********************************
*    Row   * Instance *    events *
***********************************
*        0 *        0 * -6.305098 *
*        0 *        1 * -1.712210 *
*        0 *        2 * 20.919681 *
*        0 *        3 *    0.0005 *
*        0 *        4 * 6.3050984 *
*        0 *        5 * 1.7122100 *
*        0 *        6 * -20.91968 *
*        0 *        7 *         0 *
*        1 *        0 * 7.2684128 *
*        1 *        1 * -39.31466 *
*        1 *        2 * 0.6788941 *
*        1 *        3 *    0.0005 *
*        1 *        4 * -7.268412 *
*        1 *        5 * 39.314666 *
*        1 *        6 * -0.678894 *
*        1 *        7 *         0 *
*        2 *        0 * 23.255274 *
*        2 *        1 * -4.155332 *
*        2 *        2 * -30.24351 *
*        2 *        3 *    0.0005 *
*        2 *        4 * -23.25527 *
*        2 *        5 * 4.1553323 *
*        2 *        6 * 30.243512 *
*        2 *        7 *         0 *
*        3 *        0 * -54.

Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> 

In [11]:
file.Write("",TFile.kOverwrite)

385

In [12]:
file.Close()

# Opening & Reading the File ~ Mirei


Pavel sent me the corrected event generation file, so I have made some changes to the file reading code. This is because the structure of how the TLorentzvectors were saved in the file, were changed. 

In [13]:
import ROOT 
import numpy as np
from ROOT import TLorentzVector

In [15]:
f=ROOT.TFile.Open("Tree.root","READ") 
tree=f.Get("t") # where e is the treename

In [16]:
leaves = tree.GetListOfLeaves()

# define dynamically a python class containing root Leaves objects
class PyListOfLeaves(dict) :
    pass

# create an istance
pyl = PyListOfLeaves()

In [17]:
for i in range(0,leaves.GetEntries() ) :
    leaf = leaves.At(i)
    name = leaf.GetName()
    print(name)
    # add dynamically attribute to my class 
    pyl.__setattr__(name,leaf)

eventsArray


In [18]:
vector_e=[]
vector_n=[]
nev=tree.GetEntries() #number of TLorentzVectors in the tree 'e'
for iev in range(0,nev) :
    tree.GetEntry(iev)
    # get values from the tree using Python class pyl which contains leaves
    # objects 
    px_e = pyl.eventsArray.GetValue(0)
    py_e = pyl.eventsArray.GetValue(1)
    pz_e = pyl.eventsArray.GetValue(2)
    Energy_e=pyl.eventsArray.GetValue(3)
    
    px_n = pyl.eventsArray.GetValue(4)
    py_n = pyl.eventsArray.GetValue(5)
    pz_n = pyl.eventsArray.GetValue(6)
    Energy_n=pyl.eventsArray.GetValue(7)
    
    vector_e.append((px_e,py_e,pz_e,Energy_e))
    vector_n.append((px_n,py_n,pz_n,Energy_n))
    
    if iev == 20:
        break
        
print("List of Tlorentz for e: \n",vector_e,"\n")

print("List of Tlorentz for n: \n",vector_n)

List of Tlorentz for e: 
 [(-6.305098439795047, -1.712210018046948, 20.919681614454312, 0.0005), (7.268412883277527, -39.31466640772783, 0.6788941582833732, 0.0005), (23.255274166335838, -4.155332326376257, -30.24351250261005, 0.0005), (-54.222550933152554, 32.618531907277834, -11.831043061554574, 0.0005), (45.43116390355864, 31.329899319651037, -22.783733714009493, 0.0005), (-7.691261243502942, 9.681734879571048, -27.21158237662999, 0.0005), (-8.738246108817874, 29.046893956952523, 28.29635099061826, 0.0005), (12.768853989304391, -3.786006677991509, 15.876594738351626, 0.0005), (0.1615892969640428, 0.11406384620664313, 2.6993768656668315, 0.0005), (-22.381291353222505, -19.38623059949777, -62.11708823266101, 0.0005)] 

List of Tlorentz for n: 
 [(6.305098439795047, 1.712210018046948, -20.919681614454312, 0.0), (-7.268412883277527, 39.31466640772783, -0.6788941582833732, 0.0), (-23.255274166335838, 4.155332326376257, 30.24351250261005, 0.0), (54.222550933152554, -32.618531907277834, 11